In [1]:
import numpy as np
import os

import random
import pandas as pd
import math
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as const


import scipy.interpolate, scipy.optimize

In [2]:
import sys

from s1s2_0311 import *
from read import *
from recoil_1223 import *
from plot_binned_detection_1d2d_0526 import *

In [3]:
from scipy.stats import poisson

In [4]:
folder2d = '../nestpy_unbinned_unbound_thrd0'

In [5]:

ER_pcles= ['pp','Be7_384', 'Be7_861', 'CNO', 'pep', 'nubb', 'Kr85', 'Rn222']
detectors = ['CJPL', 'Kamioka', 'LNGS', 'SURF', 'SNOlab']
NR_pcles = [ 'hep', 'dsnb','8B']
atm_pcles = ['atmNu_'+d+'_avg' for d in detectors]
nuclei_2d = 'Xenon'

In [6]:
det_pcles = ['CNO', 'pep']
bg_ideals_2d = ['nubb_only', 'all']
metallicities = ['high', 'low']

In [7]:

bin_S1 =np.linspace(2, 12000, 31)
bin_S2 =np.linspace(np.log10(100), np.log10(800e4), 31)


In [8]:
pcles_all = ER_pcles + NR_pcles + atm_pcles
pcles_all

['pp',
 'Be7_384',
 'Be7_861',
 'CNO',
 'pep',
 'nubb',
 'Kr85',
 'Rn222',
 'hep',
 'dsnb',
 '8B',
 'atmNu_CJPL_avg',
 'atmNu_Kamioka_avg',
 'atmNu_LNGS_avg',
 'atmNu_SURF_avg',
 'atmNu_SNOlab_avg']

In [9]:
binneddatas, effnums, eventR_high, eventR_low = binning_2dhist_1E7(pcles_all, bin_S1, bin_S2, nuclei = 'Xenon', 
                     folder_2d = folder2d, 
                   E_threshold_keV = 0, eff_type = '', corr = '', ebinding = True)


../nestpy_unbinned_unbound_thrd0/S1S2_pp.csv True
pp sim number =  10000000 
valid_dataf =  8765491
ER
else
 ../real_data_nest/pdf/pp_ER_Ebind_Xenon_pdf_high.txt
survival
ER
else
 ../real_data_nest/pdf/pp_ER_Ebind_Xenon_pdf_low.txt
survival
../nestpy_unbinned_unbound_thrd0/S1S2_Be7_384.csv True
Be7_384 sim number =  10000000 
valid_dataf =  8781428
ER
else
 ../real_data_nest/pdf/Be7_384_ER_Ebind_Xenon_pdf_high.txt
survival
ER
else
 ../real_data_nest/pdf/Be7_384_ER_Ebind_Xenon_pdf_low.txt
survival
../nestpy_unbinned_unbound_thrd0/S1S2_Be7_861.csv True
Be7_861 sim number =  10000000 
valid_dataf =  8798343
ER
else
 ../real_data_nest/pdf/Be7_861_ER_Ebind_Xenon_pdf_high.txt
survival
ER
else
 ../real_data_nest/pdf/Be7_861_ER_Ebind_Xenon_pdf_low.txt
survival
../nestpy_unbinned_unbound_thrd0/S1S2_CNO.csv True
CNO sim number =  10000000 
valid_dataf =  8799120
ER
else
 ../real_data_nest/pdf/CNO_ER_Ebind_Xenon_pdf_high.txt
survival
ER
else
 ../real_data_nest/pdf/CNO_ER_Ebind_Xenon_pdf_low.txt
s

In [10]:
binned_datas_dict = dict(zip(pcles_all, binneddatas))
pcle_dict = dict(zip(pcles_all, list(np.array([effnums, eventR_high, eventR_low]).T)))


In [11]:
pcle_dict

{'pp': array([1.00000000e+07, 3.68511835e+02, 3.71593038e+02]),
 'Be7_384': array([1.00000000e+07, 5.18492426e+00, 4.72865092e+00]),
 'Be7_861': array([1.00000000e+07, 1.35705827e+02, 1.23763714e+02]),
 'CNO': array([1.00000000e+07, 1.55177808e+01, 1.10767986e+01]),
 'pep': array([1.00000000e+07, 8.17608822e+00, 8.34642339e+00]),
 'nubb': array([10000000.        ,   134197.54712165,   134197.54712165]),
 'Kr85': array([1.00000000e+07, 6.08080838e+02, 6.08080838e+02]),
 'Rn222': array([1.00000000e+07, 1.73864699e+03, 1.73864699e+03]),
 'hep': array([1.00000000e+07, 2.70585004e+00, 2.79671814e+00]),
 'dsnb': array([1.00000060e+07, 5.46302567e-02, 5.46302567e-02]),
 '8B': array([1.00000000e+07, 9.51864881e+02, 7.82985628e+02]),
 'atmNu_CJPL_avg': array([1.00000000e+07, 3.60507424e-02, 3.60507424e-02]),
 'atmNu_Kamioka_avg': array([1.00000000e+07, 4.35975877e-02, 4.35975877e-02]),
 'atmNu_LNGS_avg': array([1.00000000e+07, 5.53093535e-02, 5.53093535e-02]),
 'atmNu_SURF_avg': array([1.000000

In [14]:

nubb_fractions = [0, 0.0001, 0.001, 0.01, 0.1, 1]
bg_ideals_2d = ['nubb_only', 'all']

In [15]:
etaijk_pd = pd.DataFrame(
                         index=[det_pcle + '_'+bg_ideal_2d+'_'+detector+'_'+metallicity+'_nubb'+str(nubb_fraction)
                                  for det_pcle in det_pcles
                                  for bg_ideal_2d in bg_ideals_2d
                                  for detector in detectors 
                                  for  metallicity in metallicities
                                  for nubb_fraction in nubb_fractions],
                        columns = ['null_sum', 'alter_sum']
                )


In [16]:
etaijk_pd

,null_sum,alter_sum
CNO_nubb_only_CJPL_high_nubb0,NaN,NaN
CNO_nubb_only_CJPL_high_nubb0.0001,NaN,NaN
CNO_nubb_only_CJPL_high_nubb0.001,NaN,NaN
CNO_nubb_only_CJPL_high_nubb0.01,NaN,NaN
CNO_nubb_only_CJPL_high_nubb0.1,NaN,NaN
...,...,...
pep_all_SNOlab_low_nubb0.0001,NaN,NaN
pep_all_SNOlab_low_nubb0.001,NaN,NaN
pep_all_SNOlab_low_nubb0.01,NaN,NaN
pep_all_SNOlab_low_nubb0.1,NaN,NaN


In [17]:
d = dict()
for bg, bg_ideal_2d in enumerate(bg_ideals_2d):   

    
    select_pcles_ER,_ = get_all_components_ER(ER_pcles, nuclei_2d, bg_ideal = bg_ideal_2d) 
    
    for detector in detectors:
        atm_pcle = ['atmNu_'+detector+'_avg']
        pcles_2d = select_pcles_ER + NR_pcles + atm_pcle
    
        for det_pcle in det_pcles:
            pcles_alter = pcles_2d
            pcles_null = [n for n in pcles_alter if n not in det_pcle]

            print('pcle_det   =', det_pcle, 
                '\npcles_alter = ', pcles_alter, 
                  '\npcles_null  = ', pcles_null)
            
            for m, metallicity in enumerate(metallicities):
                for n, nubb_fraction in enumerate(nubb_fractions):
                    binned_pcles_null_rate = get_evenlybinned_data(pcles_null, nubb_fraction, binned_datas_dict, pcle_dict, 
                        metallicity = metallicity, print_check = False, unit_eventR = 1/u.tonne/u.yr)
                    binned_pcles_alter_rate = get_evenlybinned_data(pcles_alter, nubb_fraction, binned_datas_dict, pcle_dict, 
                        metallicity = metallicity, print_check = False, unit_eventR = 1/u.tonne/u.yr)

                    null_sum_ijkappa = sum(binned_pcles_null_rate.flatten())
                    alter_sum_ijkappa = sum(binned_pcles_alter_rate.flatten())
                    
                    d[det_pcle + '_'+bg_ideal_2d+'_'+detector+'_'+metallicity+'_nubb'+str(nubb_fraction)+ '_null_i'] = binned_pcles_null_rate.flatten()
                    d[det_pcle + '_'+bg_ideal_2d+'_'+detector+'_'+metallicity+'_nubb'+str(nubb_fraction)+ '_alter_i'] = binned_pcles_alter_rate.flatten()
                    
                    
                    etaijk_pd['null_sum'][det_pcle + '_'+bg_ideal_2d+'_'+detector+'_'+metallicity+'_nubb'+str(nubb_fraction)] = null_sum_ijkappa
                    etaijk_pd['alter_sum'][det_pcle + '_'+bg_ideal_2d+'_'+detector+'_'+metallicity+'_nubb'+str(nubb_fraction)] = alter_sum_ijkappa


nubb_only
pcle_det   = CNO 
pcles_alter =  ['pp', 'Be7_384', 'Be7_861', 'CNO', 'pep', 'nubb', 'hep', 'dsnb', '8B', 'atmNu_CJPL_avg'] 
pcles_null  =  ['pp', 'Be7_384', 'Be7_861', 'pep', 'nubb', 'hep', 'dsnb', '8B', 'atmNu_CJPL_avg']
pcle_det   = pep 
pcles_alter =  ['pp', 'Be7_384', 'Be7_861', 'CNO', 'pep', 'nubb', 'hep', 'dsnb', '8B', 'atmNu_CJPL_avg'] 
pcles_null  =  ['pp', 'Be7_384', 'Be7_861', 'CNO', 'nubb', 'hep', 'dsnb', '8B', 'atmNu_CJPL_avg']
pcle_det   = CNO 
pcles_alter =  ['pp', 'Be7_384', 'Be7_861', 'CNO', 'pep', 'nubb', 'hep', 'dsnb', '8B', 'atmNu_Kamioka_avg'] 
pcles_null  =  ['pp', 'Be7_384', 'Be7_861', 'pep', 'nubb', 'hep', 'dsnb', '8B', 'atmNu_Kamioka_avg']
pcle_det   = pep 
pcles_alter =  ['pp', 'Be7_384', 'Be7_861', 'CNO', 'pep', 'nubb', 'hep', 'dsnb', '8B', 'atmNu_Kamioka_avg'] 
pcles_null  =  ['pp', 'Be7_384', 'Be7_861', 'CNO', 'nubb', 'hep', 'dsnb', '8B', 'atmNu_Kamioka_avg']
pcle_det   = CNO 
pcles_alter =  ['pp', 'Be7_384', 'Be7_861', 'CNO', 'pep', 'nubb', 'hep',

In [19]:
binned_etaijk_pd = pd.DataFrame.from_dict(d,orient='index').transpose()
binned_etaijk_pd

,CNO_nubb_only_CJPL_high_nubb0_null_i,CNO_nubb_only_CJPL_high_nubb0_alter_i,CNO_nubb_only_CJPL_high_nubb0.0001_null_i,CNO_nubb_only_CJPL_high_nubb0.0001_alter_i,CNO_nubb_only_CJPL_high_nubb0.001_null_i,CNO_nubb_only_CJPL_high_nubb0.001_alter_i,CNO_nubb_only_CJPL_high_nubb0.01_null_i,CNO_nubb_only_CJPL_high_nubb0.01_alter_i,CNO_nubb_only_CJPL_high_nubb0.1_null_i,CNO_nubb_only_CJPL_high_nubb0.1_alter_i,...,pep_all_SNOlab_low_nubb0.0001_null_i,pep_all_SNOlab_low_nubb0.0001_alter_i,pep_all_SNOlab_low_nubb0.001_null_i,pep_all_SNOlab_low_nubb0.001_alter_i,pep_all_SNOlab_low_nubb0.01_null_i,pep_all_SNOlab_low_nubb0.01_alter_i,pep_all_SNOlab_low_nubb0.1_null_i,pep_all_SNOlab_low_nubb0.1_alter_i,pep_all_SNOlab_low_nubb1_null_i,pep_all_SNOlab_low_nubb1_alter_i
0,0.426038,0.426038,0.426038,0.426038,0.426038,0.426038,0.426038,0.426038,0.426038,0.426038,...,0.350751,0.350751,0.350751,0.350751,0.350751,0.350751,0.350751,0.350751,0.350751,0.350751
1,0.099330,0.099330,0.099330,0.099330,0.099330,0.099330,0.099330,0.099330,0.099330,0.099330,...,0.081778,0.081778,0.081778,0.081778,0.081778,0.081778,0.081778,0.081778,0.081778,0.081778
2,0.889277,0.889277,0.889277,0.889277,0.889277,0.889277,0.889277,0.889277,0.889277,0.889277,...,0.732201,0.732201,0.732201,0.732201,0.732201,0.732201,0.732201,0.732201,0.732201,0.732201
3,1.390746,1.390746,1.390746,1.390746,1.390746,1.390746,1.390746,1.390746,1.390746,1.390746,...,1.145234,1.145234,1.145234,1.145234,1.145234,1.145234,1.145234,1.145234,1.145234,1.145234
4,2.761504,2.761504,2.761504,2.761504,2.761504,2.761504,2.761504,2.761504,2.761504,2.761504,...,2.274584,2.274584,2.274584,2.274584,2.274584,2.274584,2.274584,2.274584,2.274584,2.274584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
896,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
897,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
898,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [15]:
etaijk_pd

,null_sum,alter_sum
CNO_nubb_only_CJPL_high_nubb0,473.408997,487.057326
CNO_nubb_only_CJPL_high_nubb0.0001,483.873022,497.521351
CNO_nubb_only_CJPL_high_nubb0.001,578.049252,591.697581
CNO_nubb_only_CJPL_high_nubb0.01,1519.811552,1533.459881
CNO_nubb_only_CJPL_high_nubb0.1,10937.434552,10951.082881
...,...,...
pep_all_SNOlab_low_nubb0.0001,2541.212014,2548.559757
pep_all_SNOlab_low_nubb0.001,2635.388244,2642.735987
pep_all_SNOlab_low_nubb0.01,3577.150544,3584.498287
pep_all_SNOlab_low_nubb0.1,12994.773543,13002.121286


In [16]:
file_name =  'CNOpep'+'_Xe_ETAijk_regular_thrd0_S1S2binning.csv'
file_name


'CNOpep_Xe_ETAijk_regular_thrd0_S1S2binning.csv'

In [17]:
etaijk_pd.to_csv(file_name)

In [20]:
binnedfile_name =  'CNOpep'+'_Xe_binnedETAijk_regular_thrd0_S1S2binning.csv'
binnedfile_name


'CNOpep_Xe_binnedETAijk_regular_thrd0_S1S2binning.csv'

In [21]:
binned_etaijk_pd.to_csv(binnedfile_name)